In [503]:
import pandas as pd
import numpy as np
from scipy import constants
from lmfit import minimize, Parameters, report_fit
import matplotlib.pyplot as plt
%matplotlib inline  

In [504]:
GRDF = pd.read_csv("../Results/sorted_data.csv")
k = constants.value('Boltzmann constant in eV/K')
e = np.exp(1)

np.random.seed(111)  # set random seed for repeatability


In [505]:
################################################################################
# cubic_vals()
################################################################################

def cubic_vals(id, df):
    """returns in a dictionary x, y and starting values for fitting cubic model"""

    vals = {'NewID' : id,
            'xVals' : np.asarray(df.UsedTemp[df.NewID == id]),
            'yVals' : np.asarray(df.OriginalTraitValue[df.NewID == id]),
            'a'     : 1.,
            'b'     : 2.,
            'c'     : 3.,
            'd'     : 4.}

    return vals

################################################################################
# cubic_residuals()
################################################################################

def cubic_residuals(params, x, data):
    """returns residuals of data and model with given parameters for cubic model"""

    a = params['a'].value
    b = params['b'].value
    c = params['c'].value
    d = params['d'].value

    model = a + b*x + c*x**2 + d*x**3

    return model - data

################################################################################
# cubic_model()
################################################################################

# does not have tries as all curves converge on model
def cubic_model(id, df):
    """performs non linear least square model fitting for given ids TPC on cubic model

    keyword arguments:
        id     -- specific curve id
        df     -- dataframe containg TPC data and starting values for all ids"""

    vals = cubic_vals(id, df)  # get starting values and data from values function

    xVals    = vals["xVals"]   # temperatures
    ldata    = vals["yVals"]   # corresponding trait values

    res = {'NewID'  : vals["NewID"],
           'a'      : vals["a"],
           'b'      : vals["b"],
           'c'      : vals["c"],
           'd'      : vals["d"],
           'chisqr' : [np.NaN],
           'aic'    : [np.NaN],  # will test on each try for improvment
           'bic'    : [np.NaN]}

    # add parameters
    params = Parameters()
    params.add('a', value = vals["a"])
    params.add('b', value = vals["b"])
    params.add('c', value = vals["c"])
    params.add('d', value = vals["d"])

    # minimize
    out = minimize(cubic_residuals, params, args = (xVals, ldata))

    # save output in res convert to dataframe and return
    res = {'NewID'   : [id],
            'a'      : [out.params["a"].value],
            'b'      : [out.params["b"].value],
            'c'      : [out.params["c"].value],
            'd'      : [out.params["d"].value],
            'chisqr' : [out.chisqr],
            'aic'    : [out.aic],
            'bic'    : [out.bic]}

    res = pd.DataFrame(res)

    return res

In [506]:
def get_TSS(data):

    return sum((data - np.mean(data))**2)

def cubic_lresiduals(params, x, data):
    """returns residuals of data and model with given parameters for cubic model"""

    a = params['a'].value
    b = params['b'].value
    c = params['c'].value
    d = params['d'].value

    model = a + b*x + c*x**2 + d*x**3

    return np.log(model) - np.log(data)

In [507]:
id = 4

res = cubic_model(id, GRDF)
xVals = cubic_vals(id, GRDF)["xVals"]
yVals = cubic_vals(id, GRDF)["yVals"]


params = Parameters()
params.add('a', value = res["a"][0])
params.add('b', value = res["b"][0])
params.add('c', value = res["c"][0])
params.add('d', value = res["d"][0])

RSS = sum(cubic_residuals(params, xVals, yVals)**2)
TSS = get_TSS(yVals)

R_squared = 1 - RSS/TSS

lRSS = sum(cubic_lresiduals(params, xVals, yVals)**2)
lTSS = get_TSS(np.log(yVals))

lR_squared = 1 - (lRSS/lTSS)

print(R_squared)
print(lR_squared)

0.901342083065
nan


In [508]:
cubic_lresiduals(params, xVals, yVals)

array([        nan, -0.11465804,  0.35562441, -0.05972878, -0.05988016,
        0.02089966, -0.02401534, -0.07883133, -0.03189846,  0.47317803,
       -0.28957724])

In [509]:
laic = len(xVals)*np.log(lRSS/len(xVals)) + 2*4
laic

nan

In [510]:
res["aic"]

0   -80.014756
Name: aic, dtype: float64

In [531]:
def schlfld_vals(id, df):
    """returns in a dictionary x, y and starting values for fitting schoolfield models"""

    vals = {'NewID' : id,
            'xVals' : np.asarray(df.UsedTempK[df.NewID == id]),
            'yVals' : np.asarray(df.OTVlogged[df.NewID == id]),
            'B0'    : df.B0[df.NewID == id].iloc[0],
            'E'     : abs(df.E[df.NewID == id].iloc[0]),
            'El'    : abs(df.El[df.NewID == id].iloc[0]),
            'Eh'    : df.Eh[df.NewID == id].iloc[0],
            'Tl'    : df.Tl[df.NewID == id].iloc[0],
            'Th'    : df.Th[df.NewID == id].iloc[0]}

    return vals

################################################################################
# full_schlfld_residuals()
################################################################################

def full_schlfld_residuals(params, x, data):
    """returns residuals of data and model with given parameters for full_schlfld_model"""

    B0 = params['B0'].value
    E  = params['E'].value
    El = params['El'].value
    Eh = params['Eh'].value
    Tl = params['Tl'].value
    Th = params['Th'].value

    model = np.log((B0*e**((-E/k)*((1/x)-(1/283.15))))/(
                   1+(e**((El/k)*((1/Tl)-(1/x))))+(e**((Eh/k)*((1/Th)-(1/x))))))

    return model - data

################################################################################
# full_schlfld_model()
################################################################################

def full_schlfld_model(id, df, tries = 10, method = 1):
    """performs non linear least square model fitting for given ids TPC on full_schlfld_model

    keyword arguments:
        id     -- specific curve id
        df     -- dataframe containg TPC data and starting values for all ids
        tries  -- number of tries with randomized starting values
        method -- 1 - stops trying once model converges
                  2 - continue trying to improve fit upto tries"""

    vals = schlfld_vals(id, df)  # get starting values and data from values function

    xVals    = vals["xVals"]     # temperatures
    yVals    = vals["yVals"]     # corresponding trait values

    # res will be output - set initial values as starting values
    res = {'NewID'    : vals["NewID"],
           'B0'       : vals["B0"],
           'E'        : vals["E"],
           'El'       : vals["El"],
           'Eh'       : vals["Eh"],
           'Tl'       : vals["Tl"],
           'Th'       : vals["Th"],
           'chisqr'   : [np.NaN],
           'RSS'      : [np.NaN],
           'TSS'      : [np.NaN],
           'Rsquared' : [np.NaN],
           'aic'      : [np.NaN],  # will test on each try for improvment
           'bic'      : [np.NaN]}

    trycount = 0
    while True:

        trycount += 1  # increment trycount

        if method == 1:  # method 1 check if curve has converged or tries have run out
            if res["aic"] != [np.NaN] or trycount > tries:
                break
        elif method == 2:  # method 2 just check if tries have run out
            if trycount > tries:
                break

        try:
            # on first try use starting values
            if trycount == 1:
                params = Parameters()
                params.add('B0', value = vals["B0"], min = 0)
                params.add('E',  value = vals["E"],  min = 0)
                params.add('El', value = vals["El"], min = 0)
                params.add('Eh', value = vals["Eh"], min = 0)
                params.add('Tl', value = vals["Tl"], min = 250, max = 400)
                params.add('Th', value = vals["Th"], min = 250, max = 400)

            # on following tries starting values are randomised
            else:
                params = Parameters()
                params.add('B0', value = np.random.uniform(0, vals["B0"]*2), min = 0)
                params.add('E',  value = np.random.uniform(0, vals["E"]*2),  min = 0)
                params.add('El', value = np.random.uniform(0, vals["El"]*2), min = 0)
                params.add('Eh', value = np.random.uniform(0, vals["Eh"]*2), min = 0)
                params.add('Tl', value = vals["Tl"], min = 250, max = 400)
                params.add('Th', value = vals["Th"], min = 250, max = 400)

            # try minimize function to minimize residuals
            out = minimize(full_schlfld_residuals, params, args = (xVals, yVals))

            RSS      = sum(full_schlfld_residuals(out.params, xVals, yVals)**2)
            TSS      = get_TSS(yVals),
            Rsquared = 1 - (RSS/TSS)

            # if aic from this try is lower than previous lowest overwrite res
            # (only relevant for method == 2)
            if out.aic < res["aic"] or res["aic"] == [np.NaN]:
                res = {'NewID'    : [id],
                       'B0'       : [out.params["B0"].value],
                       'E'        : [out.params["E"].value],
                       'El'       : [out.params["El"].value],
                       'Eh'       : [out.params["Eh"].value],
                       'Tl'       : [out.params["Tl"].value],
                       'Th'       : [out.params["Th"].value],
                       'chisqr'   : [out.chisqr],
                       'RSS'      : RSS,
                       'TSS'      : TSS,
                       'Rsquared' : Rsquared,
                       'aic'      : [out.aic],
                       'bic'      : [out.bic]}
            continue

        # if it didnt converge go to next try/break if tries have run out
        except ValueError:
            continue

    # convert res to dataframe and output
    res = pd.DataFrame(res)
    return res


In [532]:
def full_schlfld_nonlogged_residuals(params, x, data):
    """returns residuals of data and model with given parameters for full_schlfld_model"""

    B0 = params['B0'].value
    E  = params['E'].value
    El = params['El'].value
    Eh = params['Eh'].value
    Tl = params['Tl'].value
    Th = params['Th'].value

    model = np.log((B0*e**((-E/k)*((1/x)-(1/283.15))))/(
                   1+(e**((El/k)*((1/Tl)-(1/x))))+(e**((Eh/k)*((1/Th)-(1/x))))))

    return np.exp(model) - np.exp(data)

In [533]:
id = 1741

res = full_schlfld_model(id, GRDF)
xVals = schlfld_vals(id, GRDF)["xVals"]
yVals = schlfld_vals(id, GRDF)["yVals"]

params = Parameters()
params.add('B0', value = res["B0"][0], min = 0)
params.add('E',  value = res["E"][0],  min = 0)
params.add('El', value = res["El"][0], min = 0)
params.add('Eh', value = res["Eh"][0], min = 0)
params.add('Tl', value = res["Tl"][0], min = 250, max = 400)
params.add('Th', value = res["Th"][0], min = 250, max = 400)


In [548]:
nl_RSS = sum(full_schlfld_nonlogged_residuals(params, xVals, yVals)**2)
nl_TSS = get_TSS(np.exp(yVals))
nl_Rsquared = 1 - (nl_RSS/nl_TSS)

print(nl_Rsquared)
print(res["Rsquared"][0])

-8.1751489261
0.864535736674


In [536]:
nl_aic = len(xVals)*np.log(nl_RSS/len(xVals)) + 2*6
cl_aic = len(xVals)*np.log(res["RSS"][0]/len(xVals)) + 2*6
ou_aic = res["aic"][0]

print(nl_aic)
print(cl_aic)
print(ou_aic)

-93.0611713291
7.13437553647
7.13437553647


In [537]:
################################################################################
# get_TSS()
################################################################################

def get_TSS(data):
    return sum((data - np.mean(data))**2)

################################################################################
# schlfld_vals()
################################################################################

def schlfld_vals(id, df):
    """returns in a dictionary x, y and starting values for fitting schoolfield models"""

    vals = {'NewID' : id,
            'xVals' : np.asarray(df.UsedTempK[df.NewID == id]),
            'yVals' : np.asarray(df.OTVlogged[df.NewID == id]),
            'B0'    : df.B0[df.NewID == id].iloc[0],
            'E'     : abs(df.E[df.NewID == id].iloc[0]),
            'El'    : abs(df.El[df.NewID == id].iloc[0]),
            'Eh'    : df.Eh[df.NewID == id].iloc[0],
            'Tl'    : df.Tl[df.NewID == id].iloc[0],
            'Th'    : df.Th[df.NewID == id].iloc[0]}

    return vals

################################################################################
# full_schlfld_residuals()
################################################################################

def full_schlfld_residuals(params, x, data):
    """returns residuals of data and model with given parameters for full_schlfld_model"""

    B0 = params['B0'].value
    E  = params['E'].value
    El = params['El'].value
    Eh = params['Eh'].value
    Tl = params['Tl'].value
    Th = params['Th'].value

    model = np.log((B0*e**((-E/k)*((1/x)-(1/283.15))))/(
                   1+(e**((El/k)*((1/Tl)-(1/x))))+(e**((Eh/k)*((1/Th)-(1/x))))))

    return model - data

################################################################################
#
################################################################################

def full_schlfld_nl_residuals(params, x, data):
    """returns non logged residuals of data and model with given parameters for full_schlfld_model"""

    B0 = params['B0'].value
    E  = params['E'].value
    El = params['El'].value
    Eh = params['Eh'].value
    Tl = params['Tl'].value
    Th = params['Th'].value

    model = np.log((B0*e**((-E/k)*((1/x)-(1/283.15))))/(
                   1+(e**((El/k)*((1/Tl)-(1/x))))+(e**((Eh/k)*((1/Th)-(1/x))))))

    return np.exp(model) - np.exp(data)


################################################################################
# full_schlfld_model()
################################################################################

def full_schlfld_model(id, df, tries = 10, method = 1):
    """performs non linear least square model fitting for given ids TPC on full_schlfld_model

    keyword arguments:
        id     -- specific curve id
        df     -- dataframe containg TPC data and starting values for all ids
        tries  -- number of tries with randomized starting values
        method -- 1 - stops trying once model converges
                  2 - continue trying to improve fit upto tries"""

    vals = schlfld_vals(id, df)  # get starting values and data from values function

    xVals    = vals["xVals"]     # temperatures
    yVals    = vals["yVals"]     # corresponding trait values

    # res will be output - set initial values as starting values
    res = {'NewID'   : vals["NewID"],
           'B0'      : vals["B0"],
           'E'       : vals["E"],
           'El'      : vals["El"],
           'Eh'      : vals["Eh"],
           'Tl'      : vals["Tl"],
           'Th'      : vals["Th"],
           'chisqr'  : [np.NaN],
           'RSS'     : [np.NaN],
           'TSS'     : [np.NaN],
           'Rsqrd'   : [np.NaN],
           'nlRSS'   : [np.NaN],
           'nlTSS'   : [np.NaN],
           'nlRsqrd' : [np.NaN],
           'nlaic'   : [np.NaN],
           'aic'     : [np.NaN],  # will test on each try for improvment
           'bic'     : [np.NaN]}

    trycount = 0
    while True:

        trycount += 1  # increment trycount

        if method == 1:  # method 1 check if curve has converged or tries have run out
            if res["aic"] != [np.NaN] or trycount > tries:
                break
        elif method == 2:  # method 2 just check if tries have run out
            if trycount > tries:
                break

        try:
            # on first try use starting values
            if trycount == 1:
                params = Parameters()
                params.add('B0', value = vals["B0"], min = 0)
                params.add('E',  value = vals["E"],  min = 0)
                params.add('El', value = vals["El"], min = 0)
                params.add('Eh', value = vals["Eh"], min = 0)
                params.add('Tl', value = vals["Tl"], min = 250, max = 400)
                params.add('Th', value = vals["Th"], min = 250, max = 400)

            # on following tries starting values are randomised
            else:
                params = Parameters()
                params.add('B0', value = np.random.uniform(0, vals["B0"]*2), min = 0)
                params.add('E',  value = np.random.uniform(0, vals["E"]*2),  min = 0)
                params.add('El', value = np.random.uniform(0, vals["El"]*2), min = 0)
                params.add('Eh', value = np.random.uniform(0, vals["Eh"]*2), min = 0)
                params.add('Tl', value = vals["Tl"], min = 250, max = 400)
                params.add('Th', value = vals["Th"], min = 250, max = 400)

            # try minimize function to minimize residuals
            out = minimize(full_schlfld_residuals, params, args = (xVals, yVals))

            RSS      = sum(full_schlfld_residuals(out.params, xVals, yVals)**2)
            TSS      = get_TSS(yVals),
            Rsquared = 1 - (RSS/TSS)

            nl_RSS   = sum(full_schlfld_nl_residuals(out.params, xVals, yVals)**2)
            nl_TSS   = get_TSS(np.exp(yVals))
            nl_Rsqrd = 1 - (nl_RSS/nl_TSS)
            nl_aic   = len(xVals)*np.log(nl_RSS/len(xVals)) + 2*6

            # if aic from this try is lower than previous lowest overwrite res
            # (only relevant for method == 2)
            if out.aic < res["aic"] or res["aic"] == [np.NaN]:
                res = {'NewID'   : [id],
                       'B0'      : [out.params["B0"].value],
                       'E'       : [out.params["E"].value],
                       'El'      : [out.params["El"].value],
                       'Eh'      : [out.params["Eh"].value],
                       'Tl'      : [out.params["Tl"].value],
                       'Th'      : [out.params["Th"].value],
                       'chisqr'  : [out.chisqr],
                       'RSS'     : RSS,
                       'TSS'     : TSS,
                       'Rsqrd'   : Rsquared,
                       'nlRSS'   : nl_RSS,
                       'nlTSS'   : nl_TSS,
                       'nlRsqrd' : nl_Rsqrd,
                       'nlaic'   : nl_aic,
                       'aic'     : [out.aic],
                       'bic'     : [out.bic]}
            continue

        # if it didnt converge go to next try/break if tries have run out
        except ValueError:
            continue

    # convert res to dataframe and output
    res = pd.DataFrame(res)
    return res


In [538]:
GRDF = pd.read_csv("../Results/sorted_data.csv") 

In [553]:
a = full_schlfld_model(1741, GRDF, method = 2, tries = 10)

print(a["Rsqrd"][0])
print(a["nlRsqrd"][0])

print(a["aic"][0])
print(a["nlaic"][0])

0.999710690425
0.984244654516
-103.547009496
-207.668506947


1

In [488]:
id     = 3
df     = GRDF
method = 1
tries  = 10

vals = schlfld_vals(id, df)  # get starting values and data from values function

xVals    = vals["xVals"]     # temperatures
yVals    = vals["yVals"]     # corresponding trait values

# res will be output - set initial values as starting values
res = {'NewID'   : vals["NewID"],
       'B0'      : vals["B0"],
       'E'       : vals["E"],
       'El'      : vals["El"],
       'Eh'      : vals["Eh"],
       'Tl'      : vals["Tl"],
       'Th'      : vals["Th"],
       'chisqr'  : [np.NaN],
       'RSS'     : [np.NaN],
       'TSS'     : [np.NaN],
       'Rsqrd'   : [np.NaN],
       'nlRSS'   : [np.NaN],
       'nlTSS'   : [np.NaN],
       'nlRsqrd' : [np.NaN],
       'nlaic'   : [np.NaN],
       'aic'     : [np.NaN],  # will test on each try for improvment
       'bic'     : [np.NaN]}

trycount = 0
while True:

    trycount += 1  # increment trycount

    if method == 1:  # method 1 check if curve has converged or tries have run out
        if res["aic"] != [np.NaN] or trycount > tries:
            break
    elif method == 2:  # method 2 just check if tries have run out
        if trycount > tries:
            break

    try:
        # on first try use starting values
        if trycount == 1:
            params = Parameters()
            params.add('B0', value = vals["B0"], min = 0)
            params.add('E',  value = vals["E"],  min = 0)
            params.add('El', value = vals["El"], min = 0)
            params.add('Eh', value = vals["Eh"], min = 0)
            params.add('Tl', value = vals["Tl"], min = 250, max = 400)
            params.add('Th', value = vals["Th"], min = 250, max = 400)

        # on following tries starting values are randomised
        else:
            params = Parameters()
            params.add('B0', value = np.random.uniform(0, vals["B0"]*2), min = 0)
            params.add('E',  value = np.random.uniform(0, vals["E"]*2),  min = 0)
            params.add('El', value = np.random.uniform(0, vals["El"]*2), min = 0)
            params.add('Eh', value = np.random.uniform(0, vals["Eh"]*2), min = 0)
            params.add('Tl', value = vals["Tl"], min = 250, max = 400)
            params.add('Th', value = vals["Th"], min = 250, max = 400)

        # try minimize function to minimize residuals
        out = minimize(full_schlfld_residuals, params, args = (xVals, yVals))

        RSS      = sum(full_schlfld_residuals(out.params, xVals, yVals)**2)
        TSS      = get_TSS(yVals)
        Rsquared = 1 - (RSS/TSS)

        nl_RSS   = sum(full_schlfld_nl_residuals(out.params, xVals, yVals)**2)
        nl_TSS   = get_TSS(np.exp(yVals))
        nl_Rsqrd = 1 - (nl_RSS/nl_TSS)
        nl_aic   = len(xVals)*np.log(nl_RSS/len(xVals)) + 2*6

        # if aic from this try is lower than previous lowest overwrite res
        # (only relevant for method == 2)
        if out.aic < res["aic"] or res["aic"] == [np.NaN]:
            res = {'NewID'   : [id],
                   'B0'      : [out.params["B0"].value],
                   'E'       : [out.params["E"].value],
                   'El'      : [out.params["El"].value],
                   'Eh'      : [out.params["Eh"].value],
                   'Tl'      : [out.params["Tl"].value],
                   'Th'      : [out.params["Th"].value],
                   'chisqr'  : [out.chisqr],
                   'RSS'     : RSS,
                   'TSS'     : TSS,
                   'Rsqrd'   : Rsquared,
                   'nlRSS'   : nl_RSS,
                   'nlTSS'   : nl_TSS,
                   'nlRsqrd' : nl_Rsqrd,
                   'nlaic'   : nl_aic,
                   'aic'     : [out.aic],
                   'bic'     : [out.bic]}
        continue

    # if it didnt converge go to next try/break if tries have run out
    except ValueError:
        continue

# convert res to dataframe and output
res = pd.DataFrame(res)

In [489]:
res

,B0,E,Eh,El,NewID,RSS,Rsqrd,TSS,Th,Tl,aic,bic,chisqr,nlRSS,nlRsqrd,nlTSS,nlaic
0,0.001013,13.636611,16.740462,13.65003,3,0.938352,0.958618,22.675103,380.052412,285.666301,-15.076779,-12.689407,0.938352,0.013874,0.601014,0.034774,-61.431827


In [491]:
params = Parameters()
params.add('B0', value = res["B0"][0], min = 0)
params.add('E',  value = res["E"][0],  min = 0)
params.add('El', value = res["El"][0], min = 0)
params.add('Eh', value = res["Eh"][0], min = 0)
params.add('Tl', value = res["Tl"][0], min = 250, max = 400)
params.add('Th', value = res["Th"][0], min = 250, max = 400)

In [492]:
RSS      = sum(full_schlfld_residuals(out.params, xVals, yVals)**2)
TSS      = get_TSS(yVals)
Rsquared = 1 - (RSS/TSS)

nl_RSS   = sum(full_schlfld_nl_residuals(out.params, xVals, yVals)**2)
nl_TSS   = get_TSS(np.exp(yVals))
nl_Rsqrd = 1 - (nl_RSS/nl_TSS)
nl_aic   = len(xVals)*np.log(nl_RSS/len(xVals)) + 2*6

In [497]:
nl_RSS   = sum(full_schlfld_nl_residuals(out.params, xVals, yVals)**2)
nl_TSS   = get_TSS(np.exp(yVals))
nl_Rsqrd = 1 - (nl_RSS/nl_TSS)

In [498]:
nl_Rsqrd

0.60101426057700924